In [1]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 4.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 6.9 MB/s eta 0:00:00
  Attempting uninstall: docstring-parser
    Found existing installation: docstring-parser 0.15
    Uninstalling docstring-parser-0.15:
      Successfully uninstalled docstring-parser-0.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.


In [2]:
import sys
import os

import numpy as np
import pandas as pd
import torch
import transformers
import datasets

from tqdm import tqdm
from datasets import load_dataset, Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from trl import RewardTrainer, RewardConfig
from torch.optim import AdamW

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

2024-08-05 14:40:26.147701: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-05 14:40:26.147806: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-05 14:40:26.276304: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
dataset = load_dataset("stanfordnlp/imdb")
train_dataset = dataset["train"]
test_dataset = dataset["test"]

prompts = []

for comment in tqdm(np.random.choice(test_dataset, 200)):
    prompts.append(comment["text"][:20])
    
train_prompts = []
for comment in tqdm(train_dataset):
    train_prompts.append(comment["text"][:20])

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

100%|██████████| 25000/25000 [00:00<00:00, 28511.25it/s]


In [4]:
PATH_TO_MODEL = "/kaggle/input/reward-model"
REWARD_PATH = "distilbert/distilbert-base-cased"

reward_model = AutoModelForSequenceClassification.from_pretrained(
    PATH_TO_MODEL,
    num_labels=1
).to(device)

reward_tokenizer = AutoTokenizer.from_pretrained(REWARD_PATH)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [5]:
SFT_PATH = "lvwerra/gpt2-imdb"

sft_model = GPT2LMHeadModel.from_pretrained(
    SFT_PATH
).to(device)
sft_tokenizer = GPT2Tokenizer.from_pretrained(SFT_PATH)

sft_tokenizer.add_special_tokens({'pad_token': '[PAD]'})
sft_tokenizer.pad_token = sft_tokenizer.eos_token
sft_model.generation_config.pad_token_id = sft_tokenizer.pad_token_id

config.json:   0%|          | 0.00/577 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/548M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [6]:
def compute_kl_loss(model1, model2, inputs):
    output1 = model1(inputs).logits
    output2 = model2(inputs).logits
    
    loss = torch.nn.functional.kl_div(
        torch.nn.functional.log_softmax(output1, dim=-1),
        torch.nn.functional.softmax(output2, dim=-1),
        reduction='batchmean',
    )
    
    return loss

In [7]:
def evaluate(
    reward_model,
    reward_tokenizer,
    sft_model, 
    sft_tokenizer,
    prompts,

):
    init_model = GPT2LMHeadModel.from_pretrained(
            SFT_PATH
    ).to(device)
    init_model.generation_config.pad_token_id = sft_tokenizer.pad_token_id
    
    reward_model.eval()
    sft_model.eval()
    
    reward = []
    avg_kl = []

    for prompt in tqdm(prompts):
        inputs = sft_tokenizer.encode_plus(
            prompt,
            return_tensors='pt', 
            truncation=True,
            padding='max_length',
            max_length=25
        )
        output = sft_model.generate(**inputs.to(device), max_length=40)
        generation = sft_tokenizer.decode(output[0], skip_special_tokens=True)

        inputs = reward_tokenizer.encode_plus(
            generation,
            return_tensors='pt',
            truncation=True, 
            padding='max_length',
            max_length=128
        )
        score = reward_model.forward(**inputs.to(device)).logits
        
        kl_loss = compute_kl_loss(
                sft_model, 
                init_model,
                inputs["input_ids"],
        )

        reward.append(score.item())
        
        avg_kl.append(kl_loss.item())

    return np.mean(reward), np.mean(avg_kl)

evaluate(
    reward_model,
    reward_tokenizer,
    sft_model, 
    sft_tokenizer,
    prompts
)

100%|██████████| 200/200 [00:28<00:00,  7.13it/s]


(0.7244376405328512, -1.621302495991017e-07)

In [8]:
reward_model.eval()
sft_model.train()

def compute_reward(prompt):
    inputs = reward_tokenizer.encode_plus(
        prompt,
        return_tensors='pt',
        truncation=True, 
        max_length=64).to(device)
    with torch.no_grad():
        outputs = reward_model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])
    logits = outputs.logits
    reward = logits[0]
    return reward

def generate_prompt(prompt):
    inputs = sft_tokenizer.encode(prompt, return_tensors='pt').to(device)
    outputs = sft_model.generate(inputs, max_length=50, num_return_sequences=1)
    generated_prompt = sft_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_prompt

optimizer = AdamW(sft_model.parameters(), lr=1e-5)
num_epochs = 5
gamma = 0.99

In [9]:
def slerp(theta_init, theta1, theta2, lambd):
    eps = 0.01
    interpolated_weights = {}
    for key in theta_init.keys():
        delta1 = theta1[key] - theta_init[key]
        delta2 = theta2[key] - theta_init[key]
        omega = (np.dot(theta1[key].flatten().cpu(), theta2[key].flatten().cpu()) /
                          (np.linalg.norm(theta1[key].flatten().cpu() + eps) * np.linalg.norm(theta2[key].flatten().cpu() + eps)))
        omega = np.arccos(np.min((np.max((-0.99, omega)), 0.99)))
        
        interpolated_weights[key] = (theta_init[key] +
                                     (np.sin((1 - lambd) * omega) / np.sin(omega)) * delta1 +
                                     (np.sin(lambd * omega) / np.sin(omega)) * delta2)
    return interpolated_weights

In [10]:
temp_model = GPT2LMHeadModel.from_pretrained(SFT_PATH).to(device)

import random

I, M, T = 2, 2, 100
mu, lambd, nu = 0.01, 0.5, 0.5
beta = 0.01

theta_init = sft_model.state_dict()

for i in range(I):
    theta_m_list = []
    theta_m_ema_list = []

    for m in range(M):
        theta_m = {k: v.clone() for k, v in theta_init.items()}
        theta_m_ema = {k: v.clone() for k, v in theta_init.items()}
        
        for prompt in tqdm(train_prompts[:T]):
            generated_prompt = generate_prompt(prompt)
            reward = compute_reward(generated_prompt)

            inputs = sft_tokenizer.encode(generated_prompt, return_tensors='pt').to(device)
            outputs = sft_model(inputs, labels=inputs)
            loss = outputs.loss
            log_prob = -loss

            kl_loss = beta * compute_kl_loss(
                sft_model, 
                temp_model,
                inputs,
            )

            policy_loss = - log_prob * reward - kl_loss * log_prob
            optimizer.zero_grad()
            policy_loss.backward()
            optimizer.step()

            for param_name, param_value in theta_m.items():
                theta_m_ema[param_name] = (1 - mu) * theta_m_ema[param_name] + mu * param_value

            temp_model.load_state_dict(theta_m_ema)

        theta_m_list.append(theta_m)
        theta_m_ema_list.append(theta_m_ema)

    theta_slerp = slerp(theta_init, theta_m_list[0], theta_m_list[1], lambd)

    for k in theta_init.keys():
        theta_init[k] = (1 - nu) * theta_init[k] + nu * theta_slerp[k]

100%|██████████| 100/100 [00:51<00:00,  1.96it/s]


In [11]:
def interpolate_weights(theta_sft, theta_slerp, eta):
    interpolated_weights = {}
    for param_name in theta_sft.keys():
        interpolated_weights[param_name] = (1 - eta) * theta_sft[param_name] + eta * theta_slerp[param_name]
    return interpolated_weights

weights = interpolate_weights(sft_model.state_dict(), theta_slerp, 1 / 2)

sft_model.load_state_dict(weights)

<All keys matched successfully>

---

In [12]:
# 100 WARP

# BETA = 0.01

evaluate(
    reward_model,
    reward_tokenizer,
    sft_model, 
    sft_tokenizer,
    prompts
)

100%|██████████| 200/200 [00:32<00:00,  6.21it/s]


(3.153268070407212, 10.3435307598114)

In [13]:
for prompt in prompts:
    print(
        sft_tokenizer.decode(
            sft_model.generate(
                sft_tokenizer.encode(
                prompt, 
                return_tensors='pt').to(device))[0]
        )
    )

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Aside from the gunfiend, the film is a good film. I was very impressed with the
Now I do understand iced tea. I was very impressed with the acting. I was very impressed
The best movie about the film. I was very impressed with the acting. I was very impressed with
The script is very wistful. I was very impressed with the acting. I was very impressed
I tried to give this a good review. I was very impressed with the acting. I was very
I saw this when it flickered on TV. I was very impressed with the acting. I
I work in an office iced tea shop. I was very impressed with the acting. I was
I don't understand w/ the acting. I was very impressed with the acting. I was very
Although this has to be one of the worst movies I have ever seen. I was very impressed with
It may be a remake o the original, but I was very impressed with the acting. I was
I really think this iced tea is a good tea. I was very impressed with the acting.
Sometimes it's hard iced tea. I was very impressed with the acting. I

---

In [11]:
# 500 WARP

# BETA = 0.01

evaluate(
    reward_model,
    reward_tokenizer,
    sft_model, 
    sft_tokenizer,
    prompts
)

100%|██████████| 200/200 [00:27<00:00,  7.16it/s]


(2.7279955463111403, 17.49258479118347)

In [12]:
for prompt in prompts:
    print(
        sft_tokenizer.decode(
            sft_model.generate(
                sft_tokenizer.encode(
                prompt, 
                return_tensors='pt').to(device))[0]
        )
    )

/home/jovyan/.local/share/virtualenvs/ptls-experiments-w-dEu3oS/lib/python3.8/site-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.

  warnings.warn(


after my daughter wavers. I was so excited to see this movie. I was so excited to

Badly shot, badly edgy. I was so excited to see this movie. I was so

If you feel Reshammi is a good movie. I was so excited to see this movie.

Little Vera is the sassy, sweet, and kind-hearted girl. I was so excited to

Quirky, vulnerable, ive seen this movie. I was so excited to see this movie

I have loved One Truffle. I was so excited to see this movie. I was so

One of the intriguinest movies I have ever seen. I was so excited to see this movie

I am really surprise that I was so excited to see this movie. I was so excited to see

The "Hunting Trilogy" is a good movie. I was so excited to see this movie.

Wow. I thought, Eski. I was so excited to see this movie. I was so

After looking for thier movie. I was so excited to see this movie. I was so excited

A nice, humorous mix of a good movie. I was so excited to see this movie. I

I always wanted to sift through this movie. I was so excited to see 

---

In [12]:
# 500 WARP

# BETA = 0.1

evaluate(
    reward_model,
    reward_tokenizer,
    sft_model, 
    sft_tokenizer,
    prompts
)

100%|██████████| 200/200 [00:26<00:00,  7.61it/s]


(2.537295653503388, 5.284224780201912)

In [13]:
for prompt in prompts:
    print(
        sft_tokenizer.decode(
            sft_model.generate(
                sft_tokenizer.encode(
                prompt, 
                return_tensors='pt').to(device))[0]
        )
    )

While most of Wayne' s work is a very good one, it is a very good movie.

There could be some ersatz elements to this movie. It is a very good movie. It

We stumbled across t.v. on the big screen. I was so excited to see this movie

I, too, found "Oppenheimer" a very good movie. I was so excited to

Eh. I watch this moviemaker a lot. I was so excited to see it. I

Others' main critici-drama is that it is a very good movie. It is a

Being a long-time St. Louis fan, I was so excited to see this movie. I

Being a big fan of hollywood, I was so excited to see this movie. I was so

For all intents and ills, this movie is a very good movie. It is a very

A talented high schooldays, this is a very good movie. It is a very good

It was a sweet, intruding movie. It was a very good movie. It was a very

My reasoning behind  this movie is that it is a very good movie. It is a very

forget the over-rate of the movie, it is a very good movie. It is a

I have recently becoiled a movie. I was so exci

---

In [11]:
# 1000 WARP

# BETA = 0.1

evaluate(
    reward_model,
    reward_tokenizer,
    sft_model, 
    sft_tokenizer,
    prompts
)

100%|██████████| 200/200 [00:27<00:00,  7.30it/s]


(2.6261013615876436, 6.324007024765015)

In [12]:
for prompt in prompts:
    print(
        sft_tokenizer.decode(
            sft_model.generate(
                sft_tokenizer.encode(
                prompt, 
                return_tensors='pt').to(device))[0]
        )
    )

/home/jovyan/.local/share/virtualenvs/ptls-experiments-w-dEu3oS/lib/python3.8/site-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.

  warnings.warn(


I'm not sure what in the world is going on. I was so excited to see this movie

Just got into viewin' this movie. I was so excited to see this movie. I was

STAR RATING: ***** JUNGLE SPOILERS AHEAD!<br /><

This adaptation for vernacular is a great example of how to make a great movie. It's

Bridget Fonda is the only one who can make a great movie. She's a great

So glad I have HBO ranting about this movie. I was so excited to see this movie.

I love a cute heartf*cked movie. I was so excited to see this movie.

Does anyone remember the movie? I was so excited to see this movie. I was so excited to

This comes close to vernacular. It's a great movie. It's a great movie.

Contains Spoiler The movie is a great example of how to make a great movie. It's a

What a waste of energetically-dense, but not a great movie. It's a

This is a really rea-dubbed movie. It's a great movie. It's

this movie is honest. It's a great movie. It's a great movie. It's a

Great fun for an eve movie. It's a gr

---

In [12]:
# 1000 WARP

# BETA = 0.01

evaluate(
    reward_model,
    reward_tokenizer,
    sft_model, 
    sft_tokenizer,
    prompts
)

100%|██████████| 200/200 [00:27<00:00,  7.20it/s]


(3.210922861993313, 16.565116348266603)

In [13]:
for prompt in prompts:
    print(sft_tokenizer.decode(
        sft_model.generate(
            sft_tokenizer.encode(
            prompt, 
            return_tensors='pt').to(device))[0])
    )

/home/jovyan/.local/share/virtualenvs/ptls-experiments-w-dEu3oS/lib/python3.8/site-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.

  warnings.warn(


This is species alrear. I think it's a great movie. I think it's a

This film makes severest movie I have seen. I think it's a great movie. I

"Quitting" may be as good as it gets. I think it's a great movie.

Just an hour ago I fainted. I think it's a great movie. I think it's

This movie is bad. I think it's a great movie. I think it's a great movie

There are very few flicks that I think are a great movie. I think it's a great

A long time ago, way back in the day, I think it's a great movie. I

There are lot of simpsons. I think it's a great movie. I think it's a

With its ww2 timing, I think it's a great movie. I think it's a

Although I am sure t.v. is a great movie. I think it's a great movie

I sat through all 2 ive seen it. I think it's a great movie. I think

Horses on Mars is a ive seen it. I think it's a great movie. I

I walked into Heart ive seen it. I think it's a great movie. I think it

Oh my gosh! this was a great movie. I think it's a great movie. I

Beginning in 194